# NOAA Lightning Analysis

## Introduction

In this notebook, I will analyze lightning strike data collected by the National Oceanic and Atmospheric Administration (NOAA). The dataset covers multiple years, including data from 1987 to 2020, as well as specific analyses for the years 2016, 2017, and 2018. My goal is to explore patterns, trends, and anomalies in lightning activity across different timeframes and geographic locations.

## Overview

To achieve this, I will perform various analyses, including:

- Calculating the total number of lightning strikes per month and visualizing them on bar graphs.
- Aggregating weekly and quarterly lightning strike data and identifying seasonal trends.
- Finding the locations with the highest number of lightning strikes in a single day and those with the most strike days.
- Examining whether certain days of the week had more lightning strikes than others.
- Merging datasets to ensure completeness and investigating missing data.
- Assigning monthly lightning strike counts to severity categories and visualizing the data using heatmaps.
- Validating data quality by checking for null values, missing dates, and ensuring geographic accuracy.
- Identifying yearly lightning strike outliers and visualizing trends using scatterplots.

By conducting these analyses, I aim to gain insights into lightning strike behavior, identify patterns, and validate the reliability of the dataset for further research.
